# Building XGBoost Model for the CoEDC dataset using Bach's features, MOTUS categories, impure windows, optimise hyperparameters with 5-fold CV, and LOSO validation on best model. 


Setting parameters for notebook so can easily change as right at top of notebook

In [ ]:
# number of CPUs to use
N_CPUS = 16

# size of windows in seconds
window_size = 5

# pure windows or majority?
PURE_WINDOWS = False

# accelerometer to be analysed; valid values are 'acg', 'axivity' and 'sens'
accelerometer = 'sens'

# any participants to exclude. Note that if processed_data_dir exists, then this will be ignored
PARTICIPANTS_TO_EXCLUDE = []

values_to_drop_before = ['Unknown']
values_to_drop_after = ['Other']

DATA_DIR = "src/dc_data/"

# name of directory to store processed data
file_prefix = 'XGB_Bach_MOTUS'

processed_data_dir = f'processed_MOTUS_{"pure" if PURE_WINDOWS else "impure"}_{accelerometer}_all_{window_size}s'

results_dir = f'results/{file_prefix}_{"pure" if PURE_WINDOWS else "impure"}_{accelerometer}_all_{window_size}s'

Import all the libraries

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
from tqdm.auto import tqdm

home_directory = os.path.expanduser("~")
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

sys.path.append(parent_dir)
data_dir = os.path.join(home_directory, DATA_DIR)
map_dir = parent_dir

import utils
import plot
import cf_matrix

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = None

# For reproducibility
np.random.seed(42)

create output directory if doesn't exist

In [ ]:
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

## Load data, map to motus, make windows and output to files

In [ ]:
def load_all_and_make_windows(datafiles):
    # Function which given a list of datafiles, loads the data and makes windows for each and concatenates and returns

    def worker(datafile):
        print("\nProcessing", datafile)
        data = utils.load_data(datafile, acc_prefix = accelerometer)
        data = utils.map_to_new_classes(data, 'annotation', os.path.join(map_dir, 'motus_class_map.json'), verbose=True)
        data = data[~data['annotation'].isin(values_to_drop_before)]
        X, Y, T = utils.make_windows(data, winsec=window_size, sample_rate=30, dropna=False, verbose=True, drop_impure=PURE_WINDOWS)
        mask = ~np.isin(Y, values_to_drop_after)
        X, Y, T = X[mask], Y[mask], T[mask]
        print(f'After dropping {values_to_drop_after}, there are {len(X)} windows left')
        pid = os.path.basename(datafile).split(".")[0]  # participant ID
        pid = np.asarray([pid] * len(X))
        return X, Y, T, pid

    results = []
    for datafile in tqdm(datafiles):
        if os.path.basename(datafile) in acc_missing:
            print("\nSkipping", datafile)
            continue
        result = worker(datafile)
        results.append(result)

    X = np.concatenate([result[0] for result in results])
    Y = np.concatenate([result[1] for result in results])
    T = np.concatenate([result[2] for result in results])
    pid = np.concatenate([result[3] for result in results])

    return X, Y, T, pid

In [ ]:
if (accelerometer == 'acg'):
    ACC_MISSING = [23, 24, 25, 34, 36]
elif (accelerometer == 'axivity'):
    raise Exception("Axivity data not tested yet")
elif (accelerometer == 'sens'):
    ACC_MISSING = []
else:
    raise Exception("Invalid accelerometer type")

acc_missing = [f'P{i:02d}.csv.gz' for i in ACC_MISSING]
acc_missing.extend([f'P{i:02d}.csv.gz' for i in PARTICIPANTS_TO_EXCLUDE])

# check if data directory exists
if not os.path.exists(data_dir):
    # raise an error
    raise Exception("Data directory does not exist. Please create it and download the data.")

if os.path.exists(processed_data_dir):
    print("Data already processed")
else:
    # ------------------------------------------
    # Process all files
    # ------------------------------------------
    datafiles =  os.path.join(data_dir, 'P[0-9][0-9].csv.gz')
    X, Y, T, pid = load_all_and_make_windows(sorted(glob(datafiles)))
    # Save arrays for future use
    os.makedirs(processed_data_dir)
    np.save(os.path.join(processed_data_dir, 'X.npy'), X)
    np.save(os.path.join(processed_data_dir, 'Y.npy'), Y)
    np.save(os.path.join(processed_data_dir, 'T.npy'), T)
    np.save(os.path.join(processed_data_dir, 'pid.npy'), pid)

Read in the data we have just written out

In [ ]:
# Load processed files
X = np.load(os.path.join(processed_data_dir, 'X.npy'), mmap_mode='r')
Y = np.load(os.path.join(processed_data_dir, 'Y.npy'))
T = np.load(os.path.join(processed_data_dir, 'T.npy'))
# pid is the participant id for each window
pid = np.load(os.path.join(processed_data_dir, 'pid.npy'))

# check if any NaNs in X
if np.isnan(X).any():
    print("NaNs found")
else:
    print("No NaNs found")

Let's count the number of window_size windows for each activity class.

In [ ]:
print('\nLabel distribution (# windows)')
print(pd.Series(Y).value_counts())

Save raw accelerometer data for each participant for some of the plotting analyses of trained model

In [ ]:
participant_accel_data = {}
for participant in np.unique(pid):
    participant_accel_data[participant] = X[pid == participant]

# Feature extraction
Using features from Bach paper - https://journals.humankinetics.com/view/journals/jmpb/5/1/article-p24.xml

In [ ]:
import bach_features

X_feats = pd.DataFrame([bach_features.bach_features(x, sample_rate=30) for x in tqdm(X)])
print(f"X_feats shape: {X_feats.shape}")

# store feature names as needed to label confusion matrix
feats_names = X_feats.columns
# convert X_feats to numpy array in preparation for training
X_feats = np.asarray(X_feats)

## Grid search to find optimised hyperparameters using 5-fold cross validation

In [ ]:
def print_test_groups_in_cv_folds(X, y, groups, cv):
    for fold, (_, test_index) in enumerate(cv.split(X, y, groups)):
        print(f"Fold_{fold} Test Participants: {np.unique(groups[test_index])}")

def check_unique_labels_in_cv_folds(X, y, groups, cv):
    unique_labels = []

    for fold, (_, test_index) in enumerate(cv.split(X, y, groups)):
        fold_unique_labels = np.unique(y[test_index])
        print(f"Fold_{fold} Unique Labels: {fold_unique_labels}")
        unique_labels.append(set(fold_unique_labels))

    # Check if all splits have the same set of unique labels
    for i in range(1, len(unique_labels)):
        if unique_labels[i] != unique_labels[0]:
            return False

    return True

In [ ]:
from sklearn.model_selection import StratifiedGroupKFold
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

scoring = ['recall_macro', 'precision_macro', 'f1_macro', 'accuracy']

# Define the parameter values that we want to search over

param_grid = {
              'eta': [0.1],
              'subsample': [0.6],
              'max_depth': [4,5, 6, 7, 8, 9, 10, 11, 12],
              'n_estimators': [50, 60, 70, 80, 90, 100, 110],
              'objective': ['multi:softprob'],
              'seed': [42]}

# Set to XGBoost model
xgb = XGBClassifier()

# XGBoost needs the labels to be integers starting from 0
le = LabelEncoder()
Y = le.fit_transform(Y)
print(le.classes_)


# Do I need normalised features? The XGB author says no - see https://github.com/dmlc/xgboost/issues/357

# Define GridSearchCV with StratifiedGroupKFold cross-validator
sgkf = StratifiedGroupKFold(n_splits=5)

print_test_groups_in_cv_folds(X_feats, Y, pid, sgkf)

if not check_unique_labels_in_cv_folds(X_feats, Y, pid, sgkf):
    raise Exception("Not all splits have the same set of unique labels")

# after discussions with Shing have decided to use recall_macro (is same as balanced accuracy) as discriminator
grid = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=sgkf, scoring=scoring, refit='recall_macro', n_jobs=N_CPUS, verbose=3, return_train_score=True)

# Fit model with data
grid.fit(X_feats, Y, groups=pid)

# Print the results
print("Best: %f using %s" % (grid.best_score_, grid.best_params_))

In [ ]:
# put results in dataframe
results = pd.DataFrame(grid.cv_results_)

# None get converted to NaN - turn them to string 'None' so easy to filter if needed
results = results.where(pd.notnull(results), 'None')

# and print out best parameters for each metric calculated
for scorer in scoring:
    # Find index of best score
    index_of_best_score = np.argmax(results['mean_test_{}'.format(scorer)])
    
    # Find best params from best score index
    best_params_for_scorer = results['params'][index_of_best_score]
    # Get corresponding fit and score times
    fit_time = results['mean_fit_time'][index_of_best_score]
    score_time = results['mean_score_time'][index_of_best_score]
    
    # Print best params for each scoring
    print(f'Score time: {score_time:.4f} - Best params for {scorer}:{best_params_for_scorer}')

print('\n{:<20} {:<20} {:<20} {:<20} {:<20}'.format('Scorer', *scoring))
for scorer in scoring:
    # Find index of best score
    index_of_best_score = np.argmax(results['mean_test_{}'.format(scorer)])

    # Calculate corresponding scores for best parameters
    best_scores_for_scorer = [results['mean_test_{}'.format(scorer)][index_of_best_score] for scorer in scoring]

    # Print best scores for each scorer
    print('{:<20} {:<20.4f} {:<20.4f} {:<20.4f} {:<20.4f}'.format(scorer, *best_scores_for_scorer))

print('\nTraining results for comparison (check for overfitting)')
for scorer in scoring:
    # Find index of best score
    index_of_best_score = np.argmax(results['mean_test_{}'.format(scorer)])

    # Calculate corresponding scores for best parameters
    best_scores_for_scorer = [results['mean_train_{}'.format(scorer)][index_of_best_score] for scorer in scoring]

    # Print best scores for each scorer
    print('{:<20} {:<20.4f} {:<20.4f} {:<20.4f} {:<20.4f}'.format(scorer, *best_scores_for_scorer))

In [ ]:
# Select columns of interest
cols_to_keep = ['params'] + \
                [col for col in results.columns if 'mean_test' in col or 'mean_train' in col]

df_results = results[cols_to_keep]
# Split the 'params' column into separate columns
expanded_params = df_results['params'].apply(pd.Series)

# Concatenate the expanded 'params' DataFrame with the original minus 'params'
df_results_expanded = pd.concat([expanded_params, df_results.drop('params', axis=1)], axis=1)

# Rename columns to remove 'mean_' prefix
df_results_expanded.rename(columns=lambda x: x.replace('mean_', ''), inplace=True)

# Display the DataFrame
df_results_expanded

In [ ]:
# Present 3D plot of search. If more than 2 variables scanned then some will need to be held at fixed value or used for coloring of points
import plotly.express as px
import plotly.graph_objects as go

# mask = results['param_class_weight'] == 'None'
# filtered = results[mask]

# create the plot with plotly
fig = px.scatter_3d(results, 
                    x='param_max_depth',
                    y='param_n_estimators',
                    z='mean_test_recall_macro',
                    color='mean_test_recall_macro',
                    color_continuous_scale='viridis')

fig.show()

fig = go.Figure(data =
    go.Contour(
        x=results['param_max_depth'],
        y=results['param_n_estimators'],
        z=results['mean_test_recall_macro'],
        colorscale='Viridis'
    ))

fig.update_layout(
    autosize=False,
    width=500,
    height=500,
)

# Add axis labels
fig.update_xaxes(title_text='Max Depth')
fig.update_yaxes(title_text='Number of Estimators')

fig.show()

## Do a LOSO fit using optimised parameters

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.base import clone
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, accuracy_score, confusion_matrix, classification_report

# Get best params
best_params = grid.best_params_

# Initialize RFC with best params
model = XGBClassifier(**best_params)

# Initialize LeaveOneGroupOut
logo = LeaveOneGroupOut()

# Get labels and put in correct order
# Note that labels in this case are numbers so need to convert back to strings, fix order and then convert back to numbers
labels = np.unique(Y)
n_labels = len(labels)
labels_as_str = le.inverse_transform(labels)
color_labels = list(plot.MOTUS_LABEL_COLORS.keys())
ordered_labels = [label for label in color_labels if label in labels_as_str]
labels_as_str = np.array(ordered_labels)
labels = le.transform(np.array(ordered_labels))
print(f'Labels as string: {labels_as_str}')
print(f'Labels: {labels}')


def train_fold(i, indices, X, X_feats, Y, pid, T):
    train_index, test_index = indices
    X_train, X_test = X_feats[train_index], X_feats[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    t_test = T[test_index]
    X_raw_test = X[test_index]

    # clone and fit the model
    local_model = clone(model)
    local_model.fit(X_train, y_train)
    
    # predict and calculate scores
    y_pred = local_model.predict(X_test)

    precision = precision_score(y_test, y_pred, average='macro', zero_division=np.nan)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=np.nan)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=np.nan)
    cohen_kappa = cohen_kappa_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    conf_mat = confusion_matrix(y_test, y_pred, labels=labels)
    test_participant = np.unique(pid[test_index])[0]
    print(f'Fold {i} Participant: {test_participant}, recall: {recall:.4f}, precision: {precision:.4f}, f1: {f1:.4f}, kappa: {cohen_kappa:.4f}, accuracy: {accuracy:.4f}')
#    X_test_pd = pd.DataFrame(X_test, columns=feats_names)
#    compare_fig = plot.plot_compare(t_test, y_test, y_pred, window_size, plot.MOTUS_LABEL_COLORS, participant=test_participant, trace=X_test_pd['xMean'])
    
    # save cm and figures to files
    pd.DataFrame(conf_mat, index=labels_as_str, columns=labels_as_str).to_csv(os.path.join(results_dir, f'{test_participant}_cm.csv'))
    with np.errstate(divide='ignore', invalid='ignore'):
        cf_matrix.make_confusion_matrix(conf_mat, sum_stats=True, categories=labels_as_str, figsize=(n_labels+1,n_labels), filename=os.path.join(results_dir, f'{test_participant}_cm.png'))

    compare_fig = plot.plot_compare(t_test, le.inverse_transform(y_test), le.inverse_transform(y_pred), window_size, plot.MOTUS_LABEL_COLORS, raw_traces=participant_accel_data[test_participant], y_extents=[-3, 3])
    compare_fig.write_html(os.path.join(results_dir, f'{test_participant}_compare.html'), include_plotlyjs='cdn', full_html=False)
    del compare_fig
    
    for label in labels_as_str:
        failure_fig = plot.plot_failure_cases(X_raw_test, le.inverse_transform(y_test), le.inverse_transform(y_pred), t_test, label, sample_rate=30, n_samples=8)
        failure_fig.write_html(os.path.join(results_dir, f'{test_participant}_ground_truth_{label}_failure.html'), include_plotlyjs='cdn', full_html=False)
        del failure_fig
    
    for label in labels_as_str:
        failure_fig = plot.plot_failure_cases(X_raw_test,le.inverse_transform(y_test), le.inverse_transform(y_pred), t_test, label, sample_rate=30, n_samples=8, swap_ground_and_predicted=True)
        failure_fig.write_html(os.path.join(results_dir, f'{test_participant}_prediction_{label}_failure.html'), include_plotlyjs='cdn', full_html=False)
        del failure_fig

    return precision, recall, f1, cohen_kappa, accuracy, conf_mat, local_model

def train_model(X, X_feats, Y, pid, T):
    indices = list(logo.split(X_feats, Y, pid))
    results = Parallel(n_jobs=N_CPUS)(delayed(train_fold)(i, index, X, X_feats, Y, pid, T) for i, index in enumerate(indices))

    # Sort the results by recall_scores (which is the third item in each tuple)
    results = sorted(results, key=lambda x: x[2])
    return results

results = train_model(X, X_feats, Y, pid, T)

In [ ]:
from prettytable import PrettyTable

# unpack results
precision_scores, recall_scores, f1_scores, cohen_kappa_scores, accuracy_scores, confusion_matrices, models = zip(*results)

# calculate mean scores
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)
mean_f1 = np.mean(f1_scores)
mean_cohen_kappa = np.mean(cohen_kappa_scores)
mean_accuracy = np.mean(accuracy_scores)

# calculate standard deviations
std_precision = np.std(precision_scores)
std_recall = np.std(recall_scores)
std_f1 = np.std(f1_scores)
std_cohen_kappa = np.std(cohen_kappa_scores)
std_accuracy = np.std(accuracy_scores)

print("\nAverages across all participants")
print(f"Recall:{mean_recall:.3f}({std_recall:.3f}) Precision:{mean_precision:.3f}({std_precision:.3f}) F1:{mean_f1:.3f}({std_f1:.3f}) Kappa:{mean_cohen_kappa:.3f}({std_cohen_kappa:.3f}) Accuracy:{mean_accuracy:.3f}({std_accuracy:.3f})")

# calculate combined confusion matrix
combined_conf_mat = np.sum(np.array(confusion_matrices), axis=0)
cf_matrix.make_confusion_matrix(combined_conf_mat, sum_stats=True, categories=labels_as_str, figsize=(n_labels+1,n_labels))

# write to files
pd.DataFrame(combined_conf_mat, index=labels_as_str, columns=labels_as_str).to_csv(os.path.join(results_dir, 'combined_cm.csv'))
with np.errstate(divide='ignore', invalid='ignore'):
    cf_matrix.make_confusion_matrix(combined_conf_mat, sum_stats=True, categories=labels_as_str, figsize=(n_labels+1,n_labels), filename=os.path.join(results_dir, 'combined_cm.png'))

precision_macro, recall_macro, f1_macro, accuracy, precision, recall, f1 = cf_matrix.calculate_metrics(combined_conf_mat)

# Create a PrettyTable object
table = PrettyTable()

# Set the column names
table.field_names = ["", "recall", "precision", "f1-score"]

# Add the metrics for each class
for i, class_ in enumerate(labels_as_str):
    table.add_row([class_, f"{recall[i]:.3f}", f"{precision[i]:.3f}", f"{f1[i]:.3f}"])

# Add an empty row
table.add_row(["", "", "", ""])

# calculate standard deviations
std_precision_macro = np.std(precision)
std_recall_macro = np.std(recall)
std_f1_macro = np.std(f1)

# Add the averages
table.add_row(["Macro Average", f"{recall_macro:.3f}({std_recall_macro:.3f})", f"{precision_macro:.3f}({std_precision_macro:.3f})", f"{f1_macro:.3f}({std_f1_macro:.3f})"])

# Add the accuracy
table.add_row(["Accuracy", f"{accuracy:.3f}", "", ""])

# Print the table
print(table)

# Get the HTML representation of the table
html_table = table.get_html_string()

# write stats to html file
with open(os.path.join(results_dir, 'metrics.html'), 'w') as file:
    file.write('<style>\n')
    file.write('table {border-collapse: collapse; font-family: sans-serif;}\n')
    file.write('th, td {border: 1px solid black; padding: 8px; text-align: left;}\n')
    file.write('th {background-color: #f2f2f2;}\n')
    file.write('</style>\n')
    file.write(html_table)

## Now fit full model and look at confusion matrix
### if not overfitted should be similar to LOSO

In [ ]:
final_model = clone(model)
final_model.fit(X_feats, Y)

Save the model as is final model fitted on all training data

In [ ]:
from joblib import dump

dump((final_model, le, labels, labels_as_str), os.path.join(results_dir, 'final_model.pkl'))

In [ ]:
from pprint import pprint
pprint(final_model.get_params())

In [ ]:
print('\nClassifier performance on all of our data')
Y_pred = final_model.predict(X_feats)
print(classification_report(Y, Y_pred, labels=labels, target_names=labels_as_str))

cm_train = confusion_matrix(Y, Y_pred, labels=labels)
pd.DataFrame(cm_train, index=labels_as_str, columns=labels_as_str).to_csv(os.path.join(results_dir, 'cm_train_all.csv'))
cf_matrix.make_confusion_matrix(cm_train, sum_stats=False, categories=labels_as_str, figsize=(n_labels+1,n_labels))

## Compress results directory using 7z format

In [ ]:
import shutil
import os
from py7zr import FILTER_LZMA2, SevenZipFile

def make_7z(output_filename, source_dir):
    filters = [{"id": FILTER_LZMA2, "preset": 9}]  # Use LZMA2 with maximum compression level
    with SevenZipFile(output_filename, mode='w', filters=filters) as z:
        z.writeall(source_dir, arcname=os.path.basename(source_dir))

# Create the 7z file in the parent directory of results_dir
sevenz_filename = os.path.join(os.path.dirname(results_dir), os.path.basename(results_dir) + '.7z')
make_7z(sevenz_filename, results_dir)

# Now it's safe to remove the original directory
shutil.rmtree(results_dir)

## What is the correlation between features in the RF model
Feature correlation gives an indication of how a pair of features are associated with one another. A pair of features with a high correlation coefficient (close to 1) tends to make an inefficient model, as we only need one of these features to extract the information required for classification. A visualisation of the correlation of all extracted features can be seen in a correlation matrix, and any pairs of features with high values can be removed from the feature extraction pipeline.

In [ ]:
X_feats_df = pd.DataFrame(X_feats, columns=feats_names)
plt.figure(figsize=(6,5))
sns.heatmap(X_feats_df.corr().abs(), cmap='coolwarm').set_title('Correlation matrix of extracted features');

## What are the most important features
Feature importance is an explainable AI technique to reveal the relative significance of individual features on model outputs. There are many different methods that can be used to determine feature importance, however in this notebook, we will use GINI importance. When looking to optimise feature extraction (less compute power and time, better model performance), features with lowest importance should be removed first.

#### GINI Importance
GINI importance is a feature importance method that can be extracted directly from the BalancedRandomForestClassifier class. More information on how exactly it works can be found [here](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-10-213).
Note: GINI importance is known to perform poorly with highly correlated features. If relying only on GINI importance to determine which features to remove, highly correlated features should be removed first.

In [ ]:
# Specify the number of top features
n = 20

# Get the indices of the top n features
top_n_indices = np.argsort(final_model.feature_importances_)[-n:]

# Select the top n feature importances and their names
top_n_importances = final_model.feature_importances_[top_n_indices]
top_n_names = np.array(feats_names)[top_n_indices]

# Plot the top n features in descending order of importance
plt.figure()
sns.barplot(x=top_n_importances, y=top_n_names, order=top_n_names[::-1]).set(title=f"Top {n} GINI Feature importance");

# Plot them all
plt.figure(figsize=(6, 30))
sns.barplot(x=final_model.feature_importances_, y=feats_names).set(title="GINI Feature importance");